<a href="https://colab.research.google.com/github/Inamul-Hassan/Basic-Recommendation-System/blob/main/Basic_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objecive

We are going to build two content based recommendation system for recommending english movies. 


*   First we will use the movies director, genre and cast to recommend similar movies.
*   Second we will use the movie plot to recommend similar movies.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
%matplotlib inline

In [2]:
data= pd.read_csv('/content/drive/MyDrive/Resume Projects/Datasets/movies_initial.csv')

## Basic Exploration

In [3]:
data.head()

,imdbID,title,year,rating,runtime,genre,released,director,writer,cast,metacritic,imdbRating,imdbVotes,poster,plot,fullplot,language,country,awards,lastupdated,type
0,1,Carmencita,1894,NOT RATED,1 min,"Documentary, Short",NaN,William K.L. Dickson,NaN,Carmencita,NaN,5.9,1032.0,https://m.media-amazon.com/images/M/MV5BMjAzND...,Performing on what looks like a small wooden s...,Performing on what looks like a small wooden s...,NaN,USA,NaN,2015-08-26 00:03:45.040000000,movie
1,5,Blacksmith Scene,1893,UNRATED,1 min,Short,1893-05-09,William K.L. Dickson,NaN,"Charles Kayser, John Ott",NaN,6.2,1189.0,NaN,Three men hammer on an anvil and pass a bottle...,A stationary camera looks at a large anvil wit...,NaN,USA,1 win.,2015-08-26 00:03:50.133000000,movie
2,3,Pauvre Pierrot,1892,NaN,4 min,"Animation, Comedy, Short",1892-10-28,�mile Reynaud,NaN,NaN,NaN,6.7,566.0,NaN,"One night, Arlequin come to see his lover Colo...","One night, Arlequin come to see his lover Colo...",NaN,France,NaN,2015-08-12 00:06:02.720000000,movie
3,8,Edison Kinetoscopic Record of a Sneeze,1894,NaN,1 min,"Documentary, Short",1894-01-09,William K.L. Dickson,NaN,Fred Ott,NaN,5.9,988.0,NaN,A man (Thomas Edison's assistant) takes a pinc...,A man (Edison's assistant) takes a pinch of sn...,NaN,USA,NaN,2015-08-10 00:21:07.127000000,movie
4,10,Employees Leaving the Lumi�re Factory,1895,NaN,1 min,"Documentary, Short",1895-03-22,Louis Lumi�re,NaN,NaN,NaN,6.9,3469.0,NaN,A man opens the big gates to the Lumi�re facto...,A man opens the big gates to the Lumi�re facto...,NaN,France,NaN,2015-08-26 00:03:56.603000000,movie


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46014 entries, 0 to 46013
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   imdbID       46014 non-null  int64  
 1   title        46014 non-null  object 
 2   year         46014 non-null  object 
 3   rating       24629 non-null  object 
 4   runtime      43377 non-null  object 
 5   genre        45615 non-null  object 
 6   released     43002 non-null  object 
 7   director     45409 non-null  object 
 8   writer       42942 non-null  object 
 9   cast         44928 non-null  object 
 10  metacritic   9401 non-null   float64
 11  imdbRating   44300 non-null  float64
 12  imdbVotes    44299 non-null  float64
 13  poster       35798 non-null  object 
 14  plot         42161 non-null  object 
 15  fullplot     41421 non-null  object 
 16  language     44899 non-null  object 
 17  country      45830 non-null  object 
 18  awards       23542 non-null  object 
 19  last

In [5]:
data.isna().sum()

imdbID             0
title              0
year               0
rating         21385
runtime         2637
genre            399
released        3012
director         605
writer          3072
cast            1086
metacritic     36613
imdbRating      1714
imdbVotes       1715
poster         10216
plot            3853
fullplot        4593
language        1115
country          184
awards         22472
lastupdated        0
type              38
dtype: int64

In [6]:
data.describe()

,imdbID,metacritic,imdbRating,imdbVotes
count,4.601400e+04,9401.000000,44300.000000,4.429900e+04
mean,8.552062e+05,56.896288,6.381454,1.180055e+04
std,1.141979e+06,17.715373,1.183113,4.934499e+04
min,1.000000e+00,1.000000,1.100000,5.000000e+00
25%,8.165750e+04,45.000000,5.800000,3.260000e+02
50%,2.524875e+05,58.000000,6.600000,9.990000e+02
75%,1.370868e+06,70.000000,7.200000,3.829000e+03
max,5.023260e+06,100.000000,9.600000,1.521105e+06


## Exploratory Data Analysis

In [ ]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [3]:
from pandas_profiling import ProfileReport

In [4]:
profile = ProfileReport(data,title="IMDB Pandas Profiling Report")

In [5]:
profile.to_file(output_file="IMDB Pandas Profiling Report.html")

Summarize dataset:   0%|          | 0/34 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
profile.to_notebook_iframe()

In [7]:
data['plot'].iloc[5]

'A group of people are standing in a straight line along the platform of a railway station, waiting for a train, which is seen coming at some distance. When the train stops at the platform, ...'

In [8]:
data['fullplot'].iloc[5]

'A group of people are standing in a straight line along the platform of a railway station, waiting for a train, which is seen coming at some distance. When the train stops at the platform, the line dissolves. The doors of the railway-cars open, and people on the platform help passengers to get off.'

## Insights

From the exploratory data analysis,


*   More than 10% of our data are. Since we are deaing with movies there is no optimal way fill in the details. thus we have to drop the missing rows.
*   This dataset contains info about movies and series. Since we are dealing with only movies we have to remove series data.
*   This data also contains information about non-english movies. Since we are building recommendatino system only for english language we have to delete all other data.(which is almost 50%)


Thus bsed on our exploration, we only need the following columns, 

  [title,genre,director,cast,fullplot,imdbRating,language,type]


## Preprocessing

### Droping unwanted columns

In [7]:
data.columns

Index(['imdbID', 'title', 'year', 'rating', 'runtime', 'genre', 'released',
       'director', 'writer', 'cast', 'metacritic', 'imdbRating', 'imdbVotes',
       'poster', 'plot', 'fullplot', 'language', 'country', 'awards',
       'lastupdated', 'type'],
      dtype='object')

In [8]:
data.drop(labels=(['imdbID','year', 'rating', 'runtime','released','writer','metacritic','imdbVotes',
       'poster','country', 'awards','lastupdated','plot']),axis=1,inplace=True)

In [9]:
data

,title,genre,director,cast,imdbRating,fullplot,language,type
0,Carmencita,"Documentary, Short",William K.L. Dickson,Carmencita,5.9,Performing on what looks like a small wooden s...,NaN,movie
1,Blacksmith Scene,Short,William K.L. Dickson,"Charles Kayser, John Ott",6.2,A stationary camera looks at a large anvil wit...,NaN,movie
2,Pauvre Pierrot,"Animation, Comedy, Short",�mile Reynaud,NaN,6.7,"One night, Arlequin come to see his lover Colo...",NaN,movie
3,Edison Kinetoscopic Record of a Sneeze,"Documentary, Short",William K.L. Dickson,Fred Ott,5.9,A man (Edison's assistant) takes a pinch of sn...,NaN,movie
4,Employees Leaving the Lumi�re Factory,"Documentary, Short",Louis Lumi�re,NaN,6.9,A man opens the big gates to the Lumi�re facto...,NaN,movie
...,...,...,...,...,...,...,...,...
46009,The New Classmate,"Drama, Family",Ashwiny Iyer Tiwari,"Swara Bhaskar, Ratna Pathak, Pankaj Tripathy",NaN,NaN,Hindi,movie
46010,Untitled Christopher Nolan Project,Sci-Fi,Christopher Nolan,NaN,NaN,Plot is being kept under wraps.,English,movie
46011,Die Turnschuh-Giganten,"Biography, Drama, War",Oliver Dommenget,"Ken Duken, Florian Bartholom�i, Torben Liebrec...",NaN,NaN,German,movie
46012,The Monster of Mangatiti,NaN,NaN,"Greta Gregory, Mark Mitchinson, Flynn Allan, H...",NaN,Heather Walsh tells her story of the time she ...,English,movie


### Missing Value

First we will deal with missing data, then we will filter our data based on type and language.

In [10]:
data.isna().sum()

title            0
genre          399
director       605
cast          1086
imdbRating    1714
fullplot      4593
language      1115
type            38
dtype: int64

#### Full Plot

In [11]:
# we will start with fullplot since it has the most number of missing values
data[data['fullplot'].isna()].index

Int64Index([   18,    19,    32,    70,    79,    82,    92,   102,   110,
              117,
            ...
            45994, 45995, 45997, 45999, 46001, 46003, 46005, 46009, 46011,
            46013],
           dtype='int64', length=4593)

In [12]:
data.drop(index=data[data['fullplot'].isna()].index,axis=0,inplace=True)

#### IMDB Rating

In [13]:
data.isna().sum()

title            0
genre          177
director       479
cast           755
imdbRating    1061
fullplot         0
language       848
type             7
dtype: int64

In [14]:
data[data['imdbRating'].isna()].index

Int64Index([  415,   422,  2354, 19851, 22717, 23458, 24687, 26182, 28227,
            28551,
            ...
            45991, 45992, 45996, 46000, 46002, 46004, 46006, 46007, 46010,
            46012],
           dtype='int64', length=1061)

In [15]:
data.drop(index=data[data['imdbRating'].isna()].index,axis=0,inplace=True)

#### Cast

In [16]:
data.isna().sum()

title           0
genre         162
director      446
cast          645
imdbRating      0
fullplot        0
language      648
type            0
dtype: int64

In [17]:
data[data['cast'].isna()].index

Int64Index([    2,     4,     5,     8,    23,    31,    37,    38,    43,
               44,
            ...
            45332, 45338, 45370, 45397, 45410, 45461, 45463, 45466, 45911,
            45949],
           dtype='int64', length=645)

In [18]:
data.drop(index=data[data['cast'].isna()].index,axis=0,inplace=True)

#### Director

In [19]:
data.isna().sum()

title           0
genre         155
director      428
cast            0
imdbRating      0
fullplot        0
language      476
type            0
dtype: int64

In [20]:
data[data['director'].isna()].index

Int64Index([ 2984,  3205,  5938,  6971,  8148,  8149,  8480,  8519,  8854,
             8855,
            ...
            44798, 44807, 44809, 45158, 45202, 45462, 45489, 45512, 45690,
            45712],
           dtype='int64', length=428)

In [21]:
data.drop(index=data[data['director'].isna()].index,axis=0,inplace=True)

#### Genre

In [22]:
data.isna().sum()

title           0
genre         149
director        0
cast            0
imdbRating      0
fullplot        0
language      469
type            0
dtype: int64

In [23]:
data[data['genre'].isna()].index

Int64Index([  488,   916,  1203,  2437,  2968,  2969,  3665,  4390,  7021,
             7829,
            ...
            45086, 45091, 45180, 45193, 45222, 45270, 45481, 45568, 45647,
            45954],
           dtype='int64', length=149)

In [24]:
data.drop(index=data[data['genre'].isna()].index,axis=0,inplace=True)

#### Language

In [25]:
data.isna().sum()

title           0
genre           0
director        0
cast            0
imdbRating      0
fullplot        0
language      453
type            0
dtype: int64

In [26]:
data[data['language'].isna()].index

Int64Index([    0,     1,     3,     6,     7,     9,    10,    11,    12,
               13,
            ...
            43900, 43992, 44053, 44130, 44329, 44491, 44518, 44754, 44918,
            44937],
           dtype='int64', length=453)

In [27]:
data.drop(index=data[data['language'].isna()].index,axis=0,inplace=True)

In [28]:
data.isna().sum()

title         0
genre         0
director      0
cast          0
imdbRating    0
fullplot      0
language      0
type          0
dtype: int64

ALL missing values are droped :)

### Deleting Series Data

In [29]:
data['type'].value_counts()

movie    38685
Name: type, dtype: int64

After droping all the missing data, we can see that our dataset doesn't contain any data of series, thus we can drop the type column.

In [30]:
data.drop('type',axis=1,inplace=True)

### Filtering based on language

In [31]:
data[data['language'] !='English'].index

Int64Index([   29,    34,    50,    62,    64,    72,    76,    77,    78,
               80,
            ...
            45787, 45801, 45805, 45810, 45879, 45881, 45946, 45982, 45998,
            46008],
           dtype='int64', length=15855)

In [32]:
data.drop(index=data[data['language'] !='English'].index,axis=0,inplace=True)

In [33]:
data.drop('language',axis=1,inplace = True)

### Removing Duplicte Data

In [34]:
data['title'].duplicated().sum()

1687

In [35]:
data[data['title'].duplicated()].index

Int64Index([  116,   584,   642,   702,   756,   809,   842,   845,   863,
              949,
            ...
            44455, 44594, 44853, 44922, 44946, 44961, 45005, 45153, 45227,
            45888],
           dtype='int64', length=1687)

In [36]:
data.drop(index=data[data['title'].duplicated()].index,axis=0,inplace=True)

### Reseting Index

In [37]:
data.reset_index(inplace=True)

In [38]:
data.drop('index',inplace=True,axis=1)

In [39]:
data

,title,genre,director,cast,imdbRating,fullplot
0,Jack and the Beanstalk,"Short, Fantasy","George S. Fleming, Edwin S. Porter",Thomas White,6.2,Porter's sequential continuity editing links s...
1,Life of an American Fireman,"Short, Action","George S. Fleming, Edwin S. Porter","Vivian Vaughan, Arthur White, James H. White",6.4,Porter's sequential continuity editing links s...
2,The Great Train Robbery,"Short, Western",Edwin S. Porter,"A.C. Abadie, Gilbert M. 'Broncho Billy' Anders...",7.4,Among the earliest existing films in American ...
3,The Land Beyond the Sunset,"Short, Drama, Fantasy",Harold M. Shaw,"Martin Fuller, Mrs. William Bechtel, Walter Ed...",7.1,"Thanks to the Fresh Air Fund, a slum child esc..."
4,Dream of a Rarebit Fiend,"Short, Fantasy","Wallace McCutcheon, Edwin S. Porter",Jack Brawn,6.7,Adapted from Winsor McCay's films and comics o...
...,...,...,...,...,...,...
21138,The Confessions of Thomas Quick,Documentary,Brian Hill,Mimmi Kandler,5.9,This chilling film tells the gripping story of...
21139,Jay Pharoah: Can I Be Me?,Comedy,Ryan Polito,Jay Pharoah,6.4,Stand up Jay Pharaoh performs his most popular...
21140,Hurricane of Fun: The Making of Wet Hot,Documentary,Amy Rice,"Elizabeth Banks, Michael Ian Black, Bradley Co...",7.1,"In 2000, David Wain and Michael Showalter set ..."
21141,Regular Show: The Movie,"Animation, Adventure, Comedy",J.G. Quintel,"Mark Hamill, David Koechner, Jason Mantzoukas,...",7.5,"To save the universe, and their friendship, Mo..."


### Formating Cast

In [40]:
data['cast'] = data['cast'].apply(lambda name : name.split(",")[0])

In [41]:
data

,title,genre,director,cast,imdbRating,fullplot
0,Jack and the Beanstalk,"Short, Fantasy","George S. Fleming, Edwin S. Porter",Thomas White,6.2,Porter's sequential continuity editing links s...
1,Life of an American Fireman,"Short, Action","George S. Fleming, Edwin S. Porter",Vivian Vaughan,6.4,Porter's sequential continuity editing links s...
2,The Great Train Robbery,"Short, Western",Edwin S. Porter,A.C. Abadie,7.4,Among the earliest existing films in American ...
3,The Land Beyond the Sunset,"Short, Drama, Fantasy",Harold M. Shaw,Martin Fuller,7.1,"Thanks to the Fresh Air Fund, a slum child esc..."
4,Dream of a Rarebit Fiend,"Short, Fantasy","Wallace McCutcheon, Edwin S. Porter",Jack Brawn,6.7,Adapted from Winsor McCay's films and comics o...
...,...,...,...,...,...,...
21138,The Confessions of Thomas Quick,Documentary,Brian Hill,Mimmi Kandler,5.9,This chilling film tells the gripping story of...
21139,Jay Pharoah: Can I Be Me?,Comedy,Ryan Polito,Jay Pharoah,6.4,Stand up Jay Pharaoh performs his most popular...
21140,Hurricane of Fun: The Making of Wet Hot,Documentary,Amy Rice,Elizabeth Banks,7.1,"In 2000, David Wain and Michael Showalter set ..."
21141,Regular Show: The Movie,"Animation, Adventure, Comedy",J.G. Quintel,Mark Hamill,7.5,"To save the universe, and their friendship, Mo..."


### Formating Genre

In [42]:
data['genre'] = data['genre'].apply(lambda name : " ".join(name.split(",")))

In [43]:
data

,title,genre,director,cast,imdbRating,fullplot
0,Jack and the Beanstalk,Short Fantasy,"George S. Fleming, Edwin S. Porter",Thomas White,6.2,Porter's sequential continuity editing links s...
1,Life of an American Fireman,Short Action,"George S. Fleming, Edwin S. Porter",Vivian Vaughan,6.4,Porter's sequential continuity editing links s...
2,The Great Train Robbery,Short Western,Edwin S. Porter,A.C. Abadie,7.4,Among the earliest existing films in American ...
3,The Land Beyond the Sunset,Short Drama Fantasy,Harold M. Shaw,Martin Fuller,7.1,"Thanks to the Fresh Air Fund, a slum child esc..."
4,Dream of a Rarebit Fiend,Short Fantasy,"Wallace McCutcheon, Edwin S. Porter",Jack Brawn,6.7,Adapted from Winsor McCay's films and comics o...
...,...,...,...,...,...,...
21138,The Confessions of Thomas Quick,Documentary,Brian Hill,Mimmi Kandler,5.9,This chilling film tells the gripping story of...
21139,Jay Pharoah: Can I Be Me?,Comedy,Ryan Polito,Jay Pharoah,6.4,Stand up Jay Pharaoh performs his most popular...
21140,Hurricane of Fun: The Making of Wet Hot,Documentary,Amy Rice,Elizabeth Banks,7.1,"In 2000, David Wain and Michael Showalter set ..."
21141,Regular Show: The Movie,Animation Adventure Comedy,J.G. Quintel,Mark Hamill,7.5,"To save the universe, and their friendship, Mo..."


### Formating Directors

In [44]:
data['director'] = data['director'].apply(lambda name : " ".join(name.split(",")))

In [45]:
data

,title,genre,director,cast,imdbRating,fullplot
0,Jack and the Beanstalk,Short Fantasy,George S. Fleming Edwin S. Porter,Thomas White,6.2,Porter's sequential continuity editing links s...
1,Life of an American Fireman,Short Action,George S. Fleming Edwin S. Porter,Vivian Vaughan,6.4,Porter's sequential continuity editing links s...
2,The Great Train Robbery,Short Western,Edwin S. Porter,A.C. Abadie,7.4,Among the earliest existing films in American ...
3,The Land Beyond the Sunset,Short Drama Fantasy,Harold M. Shaw,Martin Fuller,7.1,"Thanks to the Fresh Air Fund, a slum child esc..."
4,Dream of a Rarebit Fiend,Short Fantasy,Wallace McCutcheon Edwin S. Porter,Jack Brawn,6.7,Adapted from Winsor McCay's films and comics o...
...,...,...,...,...,...,...
21138,The Confessions of Thomas Quick,Documentary,Brian Hill,Mimmi Kandler,5.9,This chilling film tells the gripping story of...
21139,Jay Pharoah: Can I Be Me?,Comedy,Ryan Polito,Jay Pharoah,6.4,Stand up Jay Pharaoh performs his most popular...
21140,Hurricane of Fun: The Making of Wet Hot,Documentary,Amy Rice,Elizabeth Banks,7.1,"In 2000, David Wain and Michael Showalter set ..."
21141,Regular Show: The Movie,Animation Adventure Comedy,J.G. Quintel,Mark Hamill,7.5,"To save the universe, and their friendship, Mo..."


## Recommendation based on movie info

In [46]:
data['word_pool'] = data['director'] + data['cast'] + data['genre']

In [47]:
data['word_pool'][4]

'Wallace McCutcheon  Edwin S. PorterJack BrawnShort  Fantasy'

In [48]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(ngram_range=(1, 2),stop_words='english',min_df=0)
count_matrix = count.fit_transform(data['word_pool'])

In [49]:
indices = pd.Series(data.index, index=data['title'])

In [50]:
indices

title
Jack and the Beanstalk                         0
Life of an American Fireman                    1
The Great Train Robbery                        2
The Land Beyond the Sunset                     3
Dream of a Rarebit Fiend                       4
                                           ...  
The Confessions of Thomas Quick            21138
Jay Pharoah: Can I Be Me?                  21139
Hurricane of Fun: The Making of Wet Hot    21140
Regular Show: The Movie                    21141
Cosmos Laundromat                          21142
Length: 21143, dtype: int64

In [51]:
from sklearn.metrics.pairwise import cosine_similarity

In [52]:
# Function that takes in movie title as input and outputs most similar movies
def RecommendMovieByInfo(title):
    
    idx = indices[title]

    cosine_simi = cosine_similarity(count_matrix, count_matrix)

    simi_scores = list(enumerate(cosine_simi[idx]))

    simi_scores = sorted(simi_scores, key=lambda x: x[1], reverse=True)

    simi_scores = simi_scores[1:21]

    movie_indices = [i[0] for i in simi_scores]

    # Return the top 20 most similar movies
    return data['title'].iloc[movie_indices]

In [53]:
RecommendMovieByInfo('Jurassic World')

18654                          Guardians of the Galaxy
6806                   Baby: Secret of the Lost Legend
14667                                        Star Trek
63                        20,000 Leagues Under the Sea
3653                                The Phantom Planet
4457                                     Latitude Zero
4545                    Beneath the Planet of the Apes
5696                   Buck Rogers in the 25th Century
5778                        Return from Witch Mountain
5808                              Warlords of the Deep
5884                                           Mad Max
6223                                    The Last Chase
6225                       Mad Max 2: The Road Warrior
6439                                              TRON
6444                                       Battletruck
6554                    Prisoners of the Lost Universe
6573                                     Space Raiders
6579     Spacehunter: Adventures in the Forbidden Zone
6593      

In [54]:
RecommendMovieByInfo('The Mechanic')

6167                 Death Wish II
6842                  Death Wish 3
7592            Messenger of Death
9029           Cops and Robbersons
12627               Out for a Kill
17718     Guns, Girls and Gambling
7126                 Out of Bounds
12386               Half Past Dead
8491                    Deep Cover
16022               Ninja Assassin
19934             Hitman: Agent 47
840       The General Died at Dawn
2699                   Shoot First
3401                    The Gazebo
4055            Assault on a Queen
4165          Billion Dollar Brain
4176                       Caprice
4694            The Anderson Tapes
4720                   Dirty Harry
4785     Pretty Maids All in a Row
Name: title, dtype: object

In [55]:
RecommendMovieByInfo('21 Jump Street')

19271                     22 Jump Street
17059                     The Lego Movie
8579                            Motorama
16532    Smokin' Aces 2: Assassins' Ball
1103                     Ask a Policeman
1150              The Lone Wolf Spy Hunt
1224              Charlie Chan in Panama
1940                         Hue and Cry
2197                      The Sky Dragon
2261                  The Good Humor Man
2326                      Triple Trouble
4119                    One Spy Too Many
4511                        Sinful Davey
4909                                Fuzz
4926                        The Hot Rock
5119                              Shamus
5173                        Big Bad Mama
5269               The Sugarland Express
5273                      The Super Cops
5284               Uptown Saturday Night
Name: title, dtype: object

## Recommendation Based on plot

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer(stop_words='english')

tfidfv_matrix = tfidfv.fit_transform(data['fullplot'])

tfidfv_matrix.shape

(21143, 51447)

In [58]:
from sklearn.metrics.pairwise import sigmoid_kernel

cosine_simi = sigmoid_kernel(tfidfv_matrix, tfidfv_matrix)

In [59]:
# Fucntion To get recommendations
def RecommendMovieByPlot(title, cosine_simi = cosine_simi):
    idx = indices[title]

    simi_scores = list(enumerate(cosine_simi[idx]))

    simi_scores = sorted(simi_scores, key=lambda x: x[1], reverse=True)

    simi_scores = simi_scores[1:21]

    movie_indices = [i[0] for i in simi_scores]

    return data['title'].iloc[movie_indices]

In [60]:
RecommendMovieByPlot('Thor')

18576           Thor: The Dark World
17694          Thor: Tales of Asgard
19655                      I Am Thor
18129                  Almighty Thor
13611        A Little Trip to Heaven
10177    Mortal Kombat: Annihilation
13883           The Wild Blue Yonder
7150             Robotech: The Movie
9638                        Bad Moon
7573     The Incredible Hulk Returns
9479                        Lunarcop
16353                    Polar Storm
2319                    Summer Stock
3973              Crack in the World
15139                         WALL�E
1252         The Howards of Virginia
9128                    Jungleground
17168                         Kaboom
11843        The Discovery of Heaven
5425             At the Earth's Core
Name: title, dtype: object

In [61]:
RecommendMovieByPlot('The Dark Knight Rises')

12855                   Batman: Mystery of the Batwoman
9305                                     Batman Forever
18992           Batman: The Dark Knight Returns, Part 2
19678    LEGO Batman: The Movie - DC Super Heroes Unite
19359                              The Batman Shootings
21021               Lego DC Comics: Batman Be-Leaguered
8708                       Batman: Mask of the Phantasm
11647                Batman Beyond: Return of the Joker
8445                                     Batman Returns
18746            Something from Nothing: The Art of Rap
8728                                                CB4
19573                           Jimi: All Is by My Side
15080                                          The Echo
17350                        Batman: Under the Red Hood
12456                                  Biggie and Tupac
14193                            The Batman vs. Dracula
13183                          Los Angeles Plays Itself
20890                                     Batkid

In [62]:
RecommendMovieByPlot('Kung Fu Panda 2')

16410    Kung Fu Panda: Secrets of the Furious Five
17806                         Kung Fu Panda Holiday
13855                                 Kung Fu Panda
8317                     Kickboxer 2: The Road Back
18572         Kung Fu Panda: Secrets of the Masters
9130                     Kickboxer 4: The Aggressor
18076        Films of Fury: The Kung Fu Movie Movie
8071                                    Iron & Silk
3793                                        Flipper
10646                             Dynamite Brothers
5868                                  Jaguar Lives!
5553                                  Bare Knuckles
11708                      Kung Pow: Enter the Fist
4608                                The Last Valley
7580                           The Land Before Time
4064                        The Brides of Fu Manchu
6007             The Fiendish Plot of Dr. Fu Manchu
3639                            Master of the World
6168                                   Dragonslayer
12859       

In [63]:
RecommendMovieByPlot('Interstellar')

7443                    The Time Guardian
3880             The Earth Dies Screaming
12163    Shackleton's Antarctic Adventure
5227                       Idaho Transfer
5770                  Planet of Dinosaurs
3943                   The Time Travelers
7904                           Slipstream
3208                 The Story of Mankind
20954                  Journey to Le Mans
13883                The Wild Blue Yonder
17515                   Racing Extinction
2961                  Invaders from Space
9703                                   Ed
12526                          Riverworld
13563                  Aliens of the Deep
20971                           Wanderers
7710           Back to the Future Part II
12221                     Teenage Caveman
5932          The Shape of Things to Come
13503         Hellboy II: The Golden Army
Name: title, dtype: object

# Conclusion

After testing both recommendation system, the recommendation based on the movies plot works way better than the recommendation system based on movie info.

The reason for poor performance of the system based on movie info would be that we reomoved so much data from the dataset during the preprocessing phase.
Because of the it couldn't be establish proper connections between the cast and director.